In [1]:
import datetime
import requests
import json

Darksky key : 2b423d0c90e52bb805ccf6af1f305d7c ,bf11a6cc1e3fb105f612b086f6cefe19

In [12]:
def callApi(date):
    key = "bf11a6cc1e3fb105f612b086f6cefe19"
    latitude = "13.8282"
    longitude = "100.614"
    time = str(date).replace(" ", "T")
    url = "https://api.darksky.net/forecast/{}/{},{},{}".format(key, latitude, longitude, time)
    
    response = requests.get(url=url)
    return response

In [17]:
def one_month(month, year):
    date = datetime.datetime(year=year, month=month, day=30) #change day here
    nextMonth = date.month + 1
    while(date.month != nextMonth):
        yield date
        date = date + datetime.timedelta(days=1)   
        
def one_day(date):
    tomorrow = date + datetime.timedelta(days=1)
    while(date.day != tomorrow.day):
        yield date
        date = date + datetime.timedelta(minutes=15)   

In [4]:
def write_json(filename, data):
    with open("tongsdata/" + filename + ".json", "w", encoding='utf-8') as fout:
        fout.write(json.dumps(data))

___

## LOAD INTO FILES

In [18]:
#  LOAD INTO FILES

for day in one_month(10, 2018): #change month here
    day_result = {}
    filename = day.strftime("%Y-%m-%d") #change to Y-m-d because file will arrange better
    for minute in one_day(day):
        result = callApi(minute)
        day_result.update({ minute.strftime("%H:%M"): result.json()})  #  key is time of the day
    write_json(filename=filename, data=day_result)  #  filename is date    

Don't forget to download Energybalance till the lastest day

___

## Clean Data

In [2]:
import os
import pandas as pd
import json
import datetime

In [3]:
def createDayDataframe(filename):
    path = "tongsdata/"
    filedate = filename.split(".")[0].split("-")

    with open(path + filename, 'r', encoding='utf-8') as fin:
        data_json = json.loads(fin.read())

    #  Create X DataFrame
    todayData = {time: data_json[time]['currently'] for time in data_json}
    todayData_DF = pd.DataFrame(todayData).T

    #  Create y DataFrame
    energyBalanceFilename = "energyBalance/Energy_Balance_{}_{}_{}.csv".format(filedate[0], filedate[1], filedate[2]) #change due to json filename
    energyBalance = pd.read_csv(path + energyBalanceFilename, delimiter=";")

    energyBalance_DF = pd.DataFrame(index=energyBalance.iloc[:-1,0].apply(lambda x: x[2:-1]))
    energyBalance_DF['PV Power Generation'] = energyBalance['PV power generation / Mean values [W]  '].values[:-1]
    energyBalance_DF = energyBalance_DF[energyBalance_DF != ' ']
    energyBalance_DF = energyBalance_DF.dropna()
    
    #  Join X y DataFrame
    data = todayData_DF.join(energyBalance_DF)
    return data

def cleanDataframe(masterDF):
    masterData_cleaned = masterDF.reset_index()
    masterData_cleaned = masterData_cleaned.rename(columns={'index': 'time_of_day'})
    masterData_cleaned = masterData_cleaned[masterData_cleaned['PV Power Generation'].notna()]
    masterData_cleaned['month'] = masterData_cleaned['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
    masterData_cleaned['cloudCover'] = masterData_cleaned['cloudCover'].fillna(value=masterData_cleaned['cloudCover'].mean())
    masterData_cleaned['PV Power Generation'] = masterData_cleaned['PV Power Generation'].apply(lambda x: x.replace(",", "")).astype('float')
    return masterData_cleaned

In [4]:
def featureEngineering(masterDF):
    masterDF = masterDF.drop(columns=['icon', 'summary', 'time', 'windBearing', 'windGust', 'ozone', 'precipIntensity', 'precipProbability','pressure'])
    masterDF = pd.get_dummies(masterDF, columns=['time_of_day', 'precipType', 'uvIndex', 'month']) 
    return masterDF

In [18]:
files = os.listdir("tongsdata/")

#files.remove(".DS_Store")
files.remove("energyBalance")

masterData = pd.DataFrame()

for file in files:
    r = createDayDataframe(file).loc['06:30':'18:00']
    print(file, r.shape)
    masterData = pd.concat([masterData, r], axis=0)
    
#masterData = cleanDataframe(masterData)
#masterData = featureEngineering(masterData)

2018-10-01.json (47, 15)
2018-10-02.json (47, 14)
2018-10-03.json (47, 14)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


2018-10-04.json (47, 15)
2018-10-05.json (47, 15)
2018-10-06.json (47, 15)
2018-10-07.json (47, 15)
2018-10-08.json (47, 15)
2018-10-09.json (47, 15)
2018-10-10.json (47, 15)
2018-10-11.json (47, 15)
2018-10-12.json (47, 15)
2018-10-13.json (47, 14)
2018-10-14.json (47, 15)
2018-10-15.json (47, 15)
2018-10-16.json (47, 15)
2018-10-17.json (47, 14)
2018-10-18.json (47, 14)
2018-10-19.json (47, 15)
2018-10-20.json (47, 15)
2018-10-21.json (47, 15)
2018-10-22.json (47, 15)
2018-10-23.json (47, 15)
2018-10-24.json (47, 14)
2018-10-25.json (47, 15)
2018-10-26.json (47, 15)
2018-10-27.json (47, 15)
2018-10-28.json (47, 15)
2018-10-29.json (47, 15)
2018-10-30.json (47, 15)
2018-10-31.json (47, 15)
2018-11-01.json (47, 15)
2018-11-02.json (47, 13)
2018-11-03.json (47, 15)
2018-11-04.json (47, 14)
2018-11-05.json (47, 14)
2018-11-06.json (47, 15)
2018-11-07.json (47, 14)
2018-11-08.json (47, 15)
2018-11-09.json (47, 14)
2018-11-10.json (47, 15)
2018-11-11.json (47, 14)
2018-11-12.json (47, 15)


In [19]:
pd.set_option('display.max_columns', 100)
masterData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4747 entries, 06:30 to 18:00
Data columns (total 19 columns):
PV Power Generation    4652 non-null object
apparentTemperature    4730 non-null object
cloudCover             4375 non-null object
dewPoint               4730 non-null object
humidity               4730 non-null object
icon                   4730 non-null object
ozone                  204 non-null object
precipIntensity        204 non-null object
precipProbability      204 non-null object
precipType             4580 non-null object
pressure               204 non-null object
summary                4730 non-null object
temperature            4730 non-null object
time                   4747 non-null object
uvIndex                4389 non-null object
visibility             4726 non-null object
windBearing            4725 non-null object
windGust               1034 non-null object
windSpeed              4728 non-null object
dtypes: object(19)
memory usage: 741.7+ KB


In [6]:
masterData.shape

(4652, 70)

In [17]:
pd.set_option('display.max_columns', 100)
masterData

,PV Power Generation,apparentTemperature,cloudCover,dewPoint,humidity,temperature,visibility,windSpeed,time_of_day_06:30,time_of_day_06:45,time_of_day_07:00,time_of_day_07:15,time_of_day_07:30,time_of_day_07:45,time_of_day_08:00,time_of_day_08:15,time_of_day_08:30,time_of_day_08:45,time_of_day_09:00,time_of_day_09:15,time_of_day_09:30,time_of_day_09:45,time_of_day_10:00,time_of_day_10:15,time_of_day_10:30,time_of_day_10:45,time_of_day_11:00,time_of_day_11:15,time_of_day_11:30,time_of_day_11:45,time_of_day_12:00,time_of_day_12:15,time_of_day_12:30,time_of_day_12:45,time_of_day_13:00,time_of_day_13:15,time_of_day_13:30,time_of_day_13:45,time_of_day_14:00,time_of_day_14:15,time_of_day_14:30,time_of_day_14:45,time_of_day_15:00,time_of_day_15:15,time_of_day_15:30,time_of_day_15:45,time_of_day_16:00,time_of_day_16:15,time_of_day_16:30,time_of_day_16:45,time_of_day_17:00,time_of_day_17:15,time_of_day_17:30,time_of_day_17:45,time_of_day_18:00,precipType_rain,uvIndex_0,uvIndex_1,uvIndex_2,uvIndex_3,uvIndex_4,uvIndex_5,uvIndex_6,uvIndex_7,uvIndex_8,uvIndex_9,month_1,month_10,month_11,month_12
0,10.0,84.52,0.26,75.89,0.88,79.67,6.22,4.43,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,73.0,85.92,0.22,76.07,0.88,80.11,6.22,4.26,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,163.0,86.84,0.19,76.25,0.87,80.55,6.22,4.09,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,438.0,87.71,0.19,76.34,0.86,81.01,6.22,4.41,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,650.0,88.6,0.19,76.43,0.85,81.47,6.22,4.74,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
5,854.0,89.57,0.19,76.51,0.84,81.93,6.22,5.07,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
6,1423.0,90.51,0.19,76.59,0.83,82.39,6.22,5.42,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
7,1514.0,92.2,0.19,76.62,0.8,83.28,6.22,5.64,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8,1139.0,93.78,0.19,76.64,0.78,84.18,6.22,6.15,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
9,1785.0,95.27,0.19,76.62,0.76,85.07,6.22,6.89,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0


Y
    * 'PV Power Generation'

X
    * 'time_of_day'         ->  one-hot
    * 'apparentTemperature' ->  OK
    * 'cloudCover'          ->  OK 
    * 'dewPoint'            ->  OK
    * 'humidity'            ->  OK
    * 'icon'                ->  drop
    * 'precipType'          ->  one-hot
    * 'summary'             ->  drop
    * 'temperature'         ->  OK
    * 'time'                ->  drop
    * 'uvIndex'             ->  one-hot
    * 'visibility'          ->  OK
    * 'windBearing'         ->  drop (NN NE EE ...)
    * 'windGust'            ->  drop (what is wind gust?)
    * 'windSpeed'           ->  OK
    * 'month'               ->  one-hot (Should do season or quarter?)
    * 'ozone'               ->  drop (why less data)
    * 'precipIntensity'     ->  drop (why less data)
    * 'precipProbability'   ->  drop (why less data)
    * 'pressure'            ->  drop (why less data)

___

In [7]:
features = list(masterData.drop(columns=['PV Power Generation']).columns)

In [15]:
masterData = masterData.dropna()

X = masterData.drop(columns=['PV Power Generation']).astype('float').values
y = masterData['PV Power Generation'].astype('float').values


In [16]:
X

array([[84.52,  0.26, 75.89, ...,  1.  ,  0.  ,  0.  ],
       [85.92,  0.22, 76.07, ...,  1.  ,  0.  ,  0.  ],
       [86.84,  0.19, 76.25, ...,  1.  ,  0.  ,  0.  ],
       ...,
       [93.73,  0.62, 75.34, ...,  0.  ,  0.  ,  0.  ],
       [93.07,  0.62, 75.25, ...,  0.  ,  0.  ,  0.  ],
       [92.42,  0.62, 75.17, ...,  0.  ,  0.  ,  0.  ]])

In [7]:
masterData.to_csv("masterData_11.csv")

In [9]:
X.shape

(4631, 69)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR = LR.fit(X_train, y_train)

y_hat = LR.predict(X_test)

In [9]:
from sklearn.metrics import mean_squared_error, r2_score
import math

math.sqrt(mean_squared_error(y_test, y_hat))

411.74412280467203

In [13]:
result = pd.DataFrame()
result['Actual'] = y_test
result['Prediction'] = y_hat

In [17]:
result
result.to_csv("result2.csv")

In [11]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=20, population_size=20, verbosity=2) # more generation, pop size will make it closer,but it will take more time
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('best_model_fromoct.py')

Generation 1 - Current best internal CV score: -126881.67012471429
Generation 2 - Current best internal CV score: -126881.67012471429
Generation 3 - Current best internal CV score: -126881.67012471429
Generation 4 - Current best internal CV score: -123745.70433008226
Generation 5 - Current best internal CV score: -123745.70433008226
Generation 6 - Current best internal CV score: -122127.63561482672
Generation 7 - Current best internal CV score: -116867.31829971689
Generation 8 - Current best internal CV score: -116867.31829971689
Generation 9 - Current best internal CV score: -116867.31829971689
Generation 10 - Current best internal CV score: -116867.31829971689
Generation 11 - Current best internal CV score: -116867.31829971689
Generation 12 - Current best internal CV score: -116867.31829971689
Generation 13 - Current best internal CV score: -116867.31829971689
Generation 14 - Current best internal CV score: -116867.31829971689
Generation 15 - Current best internal CV score: -116867.3

True

### model 3 RandomForestRegressor(only 1 estimator)

In [15]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-91867.4890210794
exported_pipeline = RandomForestRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=1, min_samples_split=7, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
print(mean_squared_error(results, testing_target), 
mean_absolute_error(results, testing_target), 
r2_score(results, testing_target))

107073.3437145532 223.5930453667954 0.7747205641270071


### model 4 StackingEstimator(2 estimators) better mse, mae, r2 score

In [16]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator, ZeroCount

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-87019.59662711709
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    StackingEstimator(estimator=ExtraTreesRegressor(bootstrap=False, max_features=0.5, min_samples_leaf=3, min_samples_split=8, n_estimators=100)),
    ZeroCount(),
    RandomForestRegressor(bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=6, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
print(mean_squared_error(results, testing_target), 
mean_absolute_error(results, testing_target), 
r2_score(results, testing_target))

C:\ProgramData\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


113284.1830856502 220.15814575289576 0.7790102065154708


### Model 60days

In [22]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-89536.71170815293
exported_pipeline = RandomForestRegressor(bootstrap=False, max_features=0.5, min_samples_leaf=3, min_samples_split=2, n_estimators=100)


exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
print(mean_squared_error(results, testing_target), 
      mean_absolute_error(results, testing_target), 
      r2_score(results, testing_target))

99317.03691564882 214.35684733005058 0.795845398669993


### Model 80 days

In [20]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-119860.60669518374
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RidgeCV()),
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75, learning_rate=0.5, loss="huber", max_depth=9, max_features=0.25, min_samples_leaf=18, min_samples_split=8, n_estimators=100, subsample=0.8)),
    ExtraTreesRegressor(bootstrap=False, max_features=0.25, min_samples_leaf=8, min_samples_split=19, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
print(mean_squared_error(results, testing_target), 
      mean_absolute_error(results, testing_target), 
      r2_score(results, testing_target))

105840.27808042987 220.16045326691932 0.7860229166464124


In [19]:
from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error
mean_squared_error(results, testing_target)
mean_absolute_error(results, testing_target)
#r2_score(results, testing_target)
#mean_absolute_percentage_error(testing_target, results)

213.94625504322767

In [23]:
#from sklearn.feature_selection import SelectFromModel
featuresImportance = pd.DataFrame()
featuresImportance['Feature Name'] = features
featuresImportance['Importance'] = exported_pipeline.feature_importances_

featuresImportance.sort_values('Importance', ascending=False)

,Feature Name,Importance
55,uvIndex_0,0.349203
56,uvIndex_1,0.115020
3,humidity,0.094092
4,temperature,0.080881
1,cloudCover,0.057697
2,dewPoint,0.049052
0,apparentTemperature,0.041996
57,uvIndex_2,0.040832
6,windSpeed,0.036619
5,visibility,0.024266


In [24]:
result = pd.DataFrame()
result['Actual'] = testing_target
result['Prediction'] = results
result['Error'] = np.abs(testing_target - results)
result

,Actual,Prediction,Error
0,12.0,32.055667,20.055667
1,60.0,78.608000,18.608000
2,355.0,503.179667,148.179667
3,1087.0,1174.766365,87.766365
4,73.0,635.850000,562.850000
5,1254.0,1231.648667,22.351333
6,1537.0,1520.413476,16.586524
7,482.0,535.370265,53.370265
8,334.0,159.566985,174.433015
9,1625.0,1546.675024,78.324976


In [25]:
result.describe()

,Actual,Prediction,Error
count,1036.000000,1036.000000,1036.000000
mean,1067.855212,1062.501667,214.356847
std,775.411666,697.817720,231.127113
min,0.000000,1.431667,0.161500
25%,355.000000,422.259542,45.858250
50%,1040.500000,1088.967333,136.787167
75%,1688.250000,1671.826625,299.969292
max,3008.000000,2733.853476,1377.368530


In [19]:
result = pd.DataFrame()
result['Actual'] = testing_target
result['Prediction'] = results
result['Error'] = np.abs(testing_target - results)
result.to_csv("result_tpot_60days.csv")

___

In [394]:
edaData = masterData

In [397]:
edaData['windSpeed']

0      3.21
1      3.21
2      3.25
3      3.25
4      3.24
5      3.25
6      3.25
7      2.93
8      3.28
9      4.13
10     5.25
11     5.44
12     5.62
13     5.81
14        6
15     6.73
16     7.46
17     8.19
18     8.92
19     8.75
20     8.65
21     8.61
22     8.66
23      8.7
24     8.78
25     8.89
26     9.04
27     8.78
28     8.52
29     8.27
       ... 
439    6.56
440    6.63
441    6.73
442    6.29
443    5.88
444    5.49
445    5.13
446    5.41
447     5.7
448       6
449     6.3
450    6.89
451    7.56
452    8.27
453    9.03
454    8.41
455    7.81
456    7.26
457    6.75
458    6.99
459    7.34
460    7.77
461    8.27
462    7.85
463    7.45
464    7.08
465    6.75
466    6.73
467    6.72
468    6.73
Name: windSpeed, Length: 465, dtype: object

TEST

In [21]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100